In [2]:
import pandas as pd
import psycopg2
import sqlalchemy
from mrjob.job import MRJob

In [3]:
data_order = pd.read_csv('TR_OrderDetails.csv')
data_Product= pd.read_csv('TR_Products.csv')

In [4]:
data_order.head()

,OrderID,OrderDate,PropertyID,ProductID,Quantity
0,1,01-01-2015,17,41,1
1,2,01-01-2015,15,54,2
2,3,01-01-2015,5,71,2
3,4,01-01-2015,19,3,2
4,5,01-01-2015,12,45,3


In [5]:
data_Product.head()

,ProductID,ProductName,ProductCategory,Price
0,1,Large Towel,Housekeeping,9
1,2,Hand Towel,Housekeeping,5
2,3,Washcloth,Housekeeping,3
3,4,Shampoo,Housekeeping,40
4,5,Moisturizer,Housekeeping,40


In [6]:
data_property = pd.read_csv('TR_PropertyInfo.csv')
data_user = pd.read_csv('TR_UserInfo.csv')

In [ ]:
class AggregateTotalProduct(MRJob):

    def reducer_init(self):
        self.conn= psycopg2.connect(database= 'database', user ='postgres', password='mekanika', host='localhost', port='5432')

    def mapper(self, _, line):
        item=line.strip().split(',')
        year =item[1][-4:]
        yield item[1], int(item[4])

    def reducer(self, key, value):
        yield key, sum(values)

    def store(self,key,values):
        self.cur=self.conn.cursor()
        self.cur.execute("insert into total_product_yearly (year, total_product) values(%s, %s)",(key,values))

    def steps(self):
        return [
            MRStep(
                mapper=self.mapper,
                reducer=self.reducer
            )
            MRStep(
                mapper=self.mapper,
                reducer_init=self.reducer_init,
                reducer=self.reducer
            )
        ]
    def mapper_final(self):
        self.conn.commit()
        self.conn.close()

if __name__ == '__main__':
    AggregateTotalProduct.run()